# Trials with the Kaiser population

The goal of this notebook is to illustrate how to setup and run trials with the Kaiser population.

In [1]:
import os
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

import pandas as pd

from microsim.trials.trial_description import KaiserTrialDescription
from microsim.trials.trial_type import TrialType
from microsim.trials.trial import Trial
from microsim.trials.trial_outcome_assessor_factory import TrialOutcomeAssessorFactory
from microsim.treatment import TreatmentStrategiesType

In [2]:
pd.set_option('future.no_silent_downcasting', True) #make the calculation here future-proof

## A relatively simple Kaiser trial

In [3]:
pf=None #person filter will not be used for inclusion/exclusion, everyone is eligible
sampleSize = 300000
td = KaiserTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = sampleSize, 
                            duration = 4, 
                            treatmentStrategies = "sprint", #a simple way to specify a treatment strategy
                            nWorkers = 5, 
                            personFilters=pf,
                            wmhSpecific=True) #use WMH-specific risks for the Kaiser outcomes
td

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER

In [4]:
tr = Trial(td)
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER
Trial
	Trial completed: False

In [5]:
tr.print_covariate_distributions()

                          self=treated, unique people count=  193497              other=control, unique people count=  193807  
                          self                                                    other
                          -----------------------------------------------------   -----------------------------------------------------
                          min      0.25    med     0.75     max    mean     sd     min     0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------   -----------------------------------------------------
                    age    45.0    59.0    65.0    72.0   110.0    65.9     9.5    45.0    59.0    65.0    72.0   110.0    65.8     9.5
                    sbp    75.7   121.8   130.3   139.0   191.7   130.4    13.0    77.8   121.8   130.4   139.1   194.9   130.5    13.0
                    dbp    36.0    66.8    72.7    78.6   114.6    72.8     8.8    36.0    66.8    72.7  

In [6]:
%%time
tr.run()
tr

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Trial is completed.
CPU times: user 3min 45s, sys: 4min 40s, total: 8min 25s
Wall time: 18min 12s


Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER
Trial
	Trial completed: True

In [7]:
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()
toa

Trial Outcome Assessor
	Assessments:
		Name: death                    Analysis: logistic       
		Name: anyEvent                 Analysis: logistic       
		Name: vascularEventOrDeath     Analysis: logistic       
		Name: vascularEvent            Analysis: logistic       
		Name: qalys                    Analysis: linear         
		Name: meanGCP                  Analysis: linear         
		Name: lastGCP                  Analysis: linear         
		Name: cogEvent                 Analysis: logistic       
		Name: deathCox                 Analysis: cox            
		Name: cogEventCox              Analysis: cox            
		Name: vascularEventOrDeathCox  Analysis: cox            
		Name: strokeRR                 Analysis: relativeRisk   
		Name: miRR                     Analysis: relativeRisk   

In [8]:
tr.analyze(toa)
tr

Trial Description
	Trial type: 1
	Block factors: []
	Sample size: 300000
	Duration: 4
	Treatment strategies: ['bp']
	Number of workers: 5
	Person filters: 
	 None
	Population type: PopulationType.KAISER
Trial
	Trial completed: True
Trial results:
	Analysis: linear
	                                         Z      Intercept           Z SE         pValue
	                    qalys:            0.00           3.62           0.00           0.98
	                  meanGCP:            0.02          52.10           0.02           0.27
	                  lastGCP:            0.02          50.52           0.02           0.50
	Analysis: logistic
	                                         Z      Intercept           Z SE         pValue
	                    death:           -0.00          -2.16           0.01           0.92
	                 anyEvent:           -0.01           0.87           0.01           0.33
	     vascularEventOrDeath:           -0.00          -1.61           0.01           0.61
	  

In [9]:
tr.print_treatment_strategy_distributions()

                          self=treated, unique people count=  193497  
                          self
                          -----------------------------------------------------
                          min      0.25    med     0.75     max    mean     sd
                          -----------------------------------------------------
            bpMedsAdded
                      0   0.93
                      1   0.03
                      2   0.02
                      3   0.01
                      4   0.00


## A more complex Kaiser trial

We will create a person filter with user-specified criteria.

In [ ]:
pf = PersonFilter() 
#pf.add_filter("person", 
#              "lowCvLimit", 
#              lambda x: (CVModelRepository().select_outcome_model_for_person(x).get_risk_for_person(x, years=10)<0.075))
pf.add_filter("df",
              "noStatin",
              lambda x: x[DefaultTreatmentsType.STATIN.value]==0)

We will initialize a treatment strategy that includes a user-specified CV risk cutoff for statins added.

In [ ]:
ts = TreatmentStrategyRepository()
#ts._repository[TreatmentStrategiesType.BP.value] = SprintForLowerDbpGoalTreatment()
ts._repository[TreatmentStrategiesType.STATIN.value] = StatinTreatmentStrategy(cvRiskCutoff=0.075)

Setup the trial description.

In [ ]:
td = KaiserTrialDescription(trialType = TrialType.COMPLETELY_RANDOMIZED,
                            blockFactors=list(),
                            sampleSize = sampleSize, 
                            duration = 4, 
                            treatmentStrategies = ts, #a more complex way to specify treatment strategies
                            nWorkers = 5, 
                            personFilters=pf,
                            wmhSpecific=True) #use WMH-specific risks for the Kaiser outcomes
td

In [ ]:
tr = Trial(td)
tr

In [ ]:
tr.print_covariate_distributions()

In [ ]:
%%time
tr.run()
tr

In [ ]:
toa = TrialOutcomeAssessorFactory.get_trial_outcome_assessor()
toa

In [ ]:
tr.analyze(toa)
tr

In [ ]:
tr.print_treatment_strategy_distributions()

In [ ]:
tr.print_treatment_strategy_variables_distribution()